# Auto-Updating LLM Manager Example Notebook

This notebook demonstrates the new automatic update capabilities of the LLM Manager. These features allow the LLM Manager to:

1. **Automatically retrieve model and CRIS profile data** from AWS documentation URLs
2. **Cache the data locally** to reduce network requests
3. **Refresh data automatically** when cache expires
4. **Force updates** when needed, regardless of cache status
5. **Export profiles to JSON** for external use or analysis

These capabilities ensure that your application always has access to the latest model and CRIS profile information without manual updates.

## Setup the Environment

First, we need to ensure that the package is accessible. If you're running this notebook without installing the package, we'll add the parent directory to the Python path.

In [ ]:
import sys
import os
import logging
import time

# Add the parent directory to the Python path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Verify that we can find the package
print(f"Current working directory: {os.getcwd()}")
print(f"Parent directory added to path: {parent_dir}")
print(f"Python path: {sys.path}")

## Import Required Modules

Now let's import the LLM Manager and related modules.

In [ ]:
# Import the LLM Manager and related modules
from src import LLMManager, Fields, Roles
from src.ModelIDParser import ModelProfileCollection, DEFAULT_MODEL_IDS_URL, DEFAULT_MODEL_IDS_JSON_CACHE
from src.CRISProfileParser import CRISProfileCollection, DEFAULT_CRIS_PROFILES_URL, DEFAULT_CRIS_PROFILES_JSON_CACHE

## Configure AWS Authentication

Set your AWS CLI profile name below. This profile should have permissions to access AWS Bedrock.

In [ ]:
# Replace with your AWS CLI profile name
aws_profile = "default"

## Basic Initialization with Auto-Update

Let's initialize the LLM Manager with auto-update capabilities. We'll set up logging to see what's happening in real-time.

In [ ]:
# Configure logging to see detailed information
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize LLM Manager with auto-update capabilities
llm_manager = LLMManager(
    # Standard parameters
    profile_name=aws_profile,
    regions=["us-east-1", "us-west-2"],
    model_ids=["anthropic.claude-3-sonnet-20240229-v1:0"], 
    
    # Auto-update parameters using default URLs
    model_ids_url=DEFAULT_MODEL_IDS_URL,
    cris_profiles_url=DEFAULT_CRIS_PROFILES_URL,
    model_ids_cache_file="model_ids_cache.json",
    cris_profiles_cache_file="cris_profiles_cache.json",
    max_profile_age=86400,  # 1 day in seconds
    force_model_id_update=False,  # Use cache if available and not expired
    force_cris_profile_update=False,  # Use cache if available and not expired
    log_level=logging.INFO
)

## Exploring Model and CRIS Profile Data

Now that we've initialized the LLM Manager with auto-update capabilities, let's examine the model and CRIS profile data that was loaded. We'll start by retrieving the collections.

In [ ]:
# Get the model and CRIS profile collections
model_collection = llm_manager.get_model_profile_collection()
cris_collection = llm_manager.get_cris_profile_collection()

# Print some basic statistics
print(f"Number of models loaded: {len(model_collection.get_all_models())}")
print(f"Number of CRIS profiles loaded: {len(cris_collection.get_all_profiles())}")

# Print the timestamp when the data was collected
import datetime
model_timestamp = datetime.datetime.fromtimestamp(model_collection.timestamp)
cris_timestamp = datetime.datetime.fromtimestamp(cris_collection.timestamp)

print(f"\nModel data collected at: {model_timestamp}")
print(f"CRIS profile data collected at: {cris_timestamp}")

## Examining Model Data

Let's explore the model data in more detail. We'll look at a few sample models to see what information is available.

In [ ]:
# Get all models
all_models = model_collection.get_all_models()

# Print details for a few sample models
sample_count = 0
for model_id, model_info in all_models.items():
    if sample_count >= 5:
        break
        
    print(f"\nModel ID: {model_id}")
    print(f"  Regions: {', '.join(model_info.regions[:3])}{'...' if len(model_info.regions) > 3 else ''}")
    print(f"  Capabilities: {', '.join(model_info.capabilities)}")
    print(f"  Streaming supported: {model_info.streaming_supported}")
    
    sample_count += 1

print(f"\nShowing {sample_count} of {len(all_models)} models")

## Examining CRIS Profile Data

Now let's explore the CRIS profile data to understand what information is available.

In [ ]:
# Get all CRIS profiles
all_profiles = cris_collection.get_all_profiles()

# Print details for a few sample profiles
sample_count = 0
for profile_id, profile_info in all_profiles.items():
    if sample_count >= 5:
        break
        
    print(f"\nProfile ID: {profile_id}")
    print(f"  Profile Name: {profile_info.profile_name}")
    print(f"  Source Regions: {', '.join(profile_info.source_regions)}")
    
    # Print a sample of destination regions for the first source region
    if profile_info.source_regions:
        first_source = profile_info.source_regions[0]
        destinations = profile_info.get_destination_regions(first_source)
        print(f"  Destinations from {first_source}: {', '.join(destinations)}")
    
    sample_count += 1

print(f"\nShowing {sample_count} of {len(all_profiles)} profiles")

## Force-Updating Data

Now let's demonstrate how to force an update from the URLs, regardless of whether the cache is valid. This ensures you always have the latest data.

In [ ]:
# Initialize with force update
print("Initializing LLM Manager with forced updates...")
updated_llm_manager = LLMManager(
    profile_name=aws_profile,
    regions=["us-east-1", "us-west-2"],
    model_ids=["anthropic.claude-3-sonnet-20240229-v1:0"],
    
    # Force update both model IDs and CRIS profiles
    model_ids_cache_file="model_ids_cache_updated.json",
    cris_profiles_cache_file="cris_profiles_cache_updated.json",
    force_model_id_update=True,
    force_cris_profile_update=True,
    log_level=logging.INFO
)

# Get the updated collections
updated_model_collection = updated_llm_manager.get_model_profile_collection()
updated_cris_collection = updated_llm_manager.get_cris_profile_collection()

# Print timestamps to verify they're recent
model_timestamp = datetime.datetime.fromtimestamp(updated_model_collection.timestamp)
cris_timestamp = datetime.datetime.fromtimestamp(updated_cris_collection.timestamp)

print(f"\nUpdated model data collected at: {model_timestamp}")
print(f"Updated CRIS profile data collected at: {cris_timestamp}")

## Exporting Profile Collections to JSON

The LLM Manager provides a convenient method to export both model and CRIS profile data to JSON files. This can be useful for:

In [ ]:
# Export profile collections to JSON files
export_model_path = "exported_model_profiles.json"
export_cris_path = "exported_cris_profiles.json"

model_success, cris_success = updated_llm_manager.export_profiles_to_json(
    model_file_path=export_model_path,
    cris_file_path=export_cris_path
)

if model_success:
    print(f"Successfully exported model data to {export_model_path}")
else:
    print(f"Failed to export model data")
    
if cris_success:
    print(f"Successfully exported CRIS profile data to {export_cris_path}")
else:
    print(f"Failed to export CRIS profile data")

# Show file sizes
if os.path.exists(export_model_path):
    model_size = os.path.getsize(export_model_path) / 1024  # KB
    print(f"Model profile JSON size: {model_size:.2f} KB")
    
if os.path.exists(export_cris_path):
    cris_size = os.path.getsize(export_cris_path) / 1024  # KB
    print(f"CRIS profile JSON size: {cris_size:.2f} KB")

## Loading from Exported JSON Files

You can also load model and CRIS profile collections directly from JSON files using the static methods provided.

In [ ]:
# Load model collection from exported JSON
loaded_model_collection = ModelProfileCollection.from_json(export_model_path)
loaded_cris_collection = CRISProfileCollection.from_json(export_cris_path)

# Verify that the data was loaded correctly
print(f"Loaded {len(loaded_model_collection.get_all_models())} models from {export_model_path}")
print(f"Loaded {len(loaded_cris_collection.get_all_profiles())} CRIS profiles from {export_cris_path}")

## Using the LLM Manager with Auto-Updated Data

Now that we have the LLM Manager with auto-updated data, let's use it to interact with AWS Bedrock. We'll send a simple prompt to demonstrate that it works correctly with the auto-updated model and CRIS information.

In [ ]:
# Define a simple prompt
prompt = "What is AWS Bedrock?"
messages = [updated_llm_manager.create_text_message(prompt)]

# Add a system message
system = [updated_llm_manager.create_system_message("You are a helpful AI assistant specializing in AWS services.")]

try:
    # Send the prompt to the model
    print("Sending prompt to model...")
    response = updated_llm_manager.converse(messages=messages, system=system)
    
    # Print the response
    print("\nModel response:")
    print(response.get_content_text())
    
    # Print metadata
    print("\nMetadata:")
    print(f"Model used: {response.model_id}")
    print(f"Region used: {response.region}")
    print(f"Used CRIS: {response.is_cris}")
    print(f"Execution time: {response.execution_time:.2f} seconds")
    print(f"Total tokens: {response.get_total_tokens()}")
    
except Exception as e:
    print(f"Error: {str(e)}")

## Cache Expiration and Auto-Refresh

The LLM Manager is designed to automatically refresh the cache when it expires. Let's demonstrate how to check if a cache is expired and how the auto-refresh works.

In [ ]:
# Check if cache is expired
def check_cache_expiry(collection, max_age_seconds):
    timestamp = collection.timestamp
    current_time = int(time.time())
    age = current_time - timestamp
    is_expired = collection.is_expired(max_age_seconds)
    
    print(f"Cache age: {age} seconds ({age/3600:.2f} hours)")
    print(f"Max age: {max_age_seconds} seconds ({max_age_seconds/3600:.2f} hours)")
    print(f"Cache expired: {is_expired}")
    return is_expired

# Check our model collection
print("Checking model collection cache:")
check_cache_expiry(updated_model_collection, 3600)  # 1 hour max age

# Check with a very short expiry to simulate an expired cache
print("\nSimulating expired cache (1 second max age):")
is_expired = check_cache_expiry(updated_model_collection, 1)  # 1 second max age

if is_expired:
    print("\nCache is expired. In a real application, the LLM Manager would automatically refresh the data from the URLs.")

## Creating a Custom LLM Manager with Different Cache Settings

You can customize the caching behavior of the LLM Manager to match your application's needs. Here, we'll create an LLM Manager with a very short cache expiration (5 minutes) for demonstration purposes.

In [ ]:
# Create an LLM Manager with custom cache settings
custom_llm_manager = LLMManager(
    profile_name=aws_profile,
    regions=["us-east-1", "us-west-2"],
    model_ids=["anthropic.claude-3-sonnet-20240229-v1:0"],
    
    # Custom cache settings
    model_ids_cache_file="model_ids_custom_cache.json",
    cris_profiles_cache_file="cris_profiles_custom_cache.json",
    max_profile_age=300,  # 5 minutes in seconds
    force_model_id_update=False,
    force_cris_profile_update=False,
    log_level=logging.INFO
)

# Get the collections
custom_model_collection = custom_llm_manager.get_model_profile_collection()
custom_cris_collection = custom_llm_manager.get_cris_profile_collection()

print(f"Custom LLM Manager initialized with {len(custom_model_collection.get_all_models())} models")
print(f"Custom LLM Manager initialized with {len(custom_cris_collection.get_all_profiles())} CRIS profiles")
print(f"Cache will expire in 5 minutes (300 seconds)")

## Conclusion

In this notebook, we've demonstrated the auto-update capabilities of the LLM Manager. Key features include:

1. **Automatic retrieval** of model and CRIS profile data from AWS documentation URLs
2. **Local caching** to reduce network requests
3. **Automatic cache refresh** when data expires
4. **Force updates** when needed, regardless of cache status
5. **Export and import** of profile collections to and from JSON files

These features ensure that your applications always have up-to-date information about AWS Bedrock models and CRIS profiles, without manual updates.

The LLM Manager's auto-update system is highly configurable, allowing you to control:
- The URLs to fetch data from
- The file paths for caching data
- The maximum age of cached data
- Whether to force updates regardless of cache status

This flexibility makes it suitable for a wide range of applications, from development environments to production systems.